In [3]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer_data = load_breast_cancer()

X_data = cancer_data.data
y_label = cancer_data.target

X_train,X_test,y_train,y_test = train_test_split(X_data,y_label,test_size=0.2, random_state=0)

In [6]:
## 스태킹에 사용할 개별 알고리즘
## knn, 랜덤포레스트, 결정트리, 에이다부스트
## 이들 모델의 예측 결과를 합한 데이터 세트로 학습/예측하는 최종 모델은 로지스틱 회귀

## 개별 Ml 모델
knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100,random_state=0)
dt_clf = DecisionTreeClassifier()
adb_clf = AdaBoostClassifier(n_estimators=100)

## 스태킹으로 만들어진 데이터 세트를 학습,예측하는 최종 모델
lr_final = LogisticRegression(C=10)

In [8]:
# 개별 모델들 학습
knn_clf.fit(X_train,y_train)
rf_clf.fit(X_train,y_train)
dt_clf.fit(X_train,y_train)
adb_clf.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=None)

In [14]:
# 학습된 개별 모델들이 각자 반환하는 예측 데이터 세트를 생성하고 개별 모델의 정확도 측정
knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
adb_pred = adb_clf.predict(X_test)

print('knn_clf 정확도',accuracy_score(y_test,knn_pred))
print('rf_clf 정확도',accuracy_score(y_test,rf_pred))
print('dt_clf 정확도',accuracy_score(y_test,dt_pred))
print('adb_clf 정확도',accuracy_score(y_test,adb_pred))

knn_clf 정확도 0.9210526315789473
rf_clf 정확도 0.9649122807017544
dt_clf 정확도 0.9035087719298246
adb_clf 정확도 0.956140350877193


In [19]:
# 개별 알고리즘으로부터 예측된 컬럼 라벨을 옆으로 붙여 피처 값으로 만들어, 최종 "메타데이터"로 로지스틱 모델로 학습/예측

pred = np.array([knn_pred,rf_pred,dt_pred,adb_pred])
print(pred.shape)
# transpose를 이용해 행과 열의 위치 교환. 칼럼 레벨로 각 알고리즘의 예측 결과를 피처로 만듦
pred = np.transpose(pred)
print(pred.shape)

(4, 114)
(114, 4)


In [22]:
lr_final.fit(pred,y_test) # 개별알고리즘들의 예측값과 "y_test" 로 학습. # 단, 과적합위험
final = lr_final.predict(pred)

print('최종 모델의 예측 정확도 : ',accuracy_score(y_test,final))

최종 모델의 예측 정확도 :  0.9736842105263158
